## Projet BiLSTM

Liste des imports

In [2]:
##Si l'archive n'est pas présente, on la télécharge et on la dézippe
import os
if not os.path.exists("cornell_movie_dialogs_corpus.zip"):
  print("Downloading data (234 MB)")
  !wget http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
  !unzip cornell_movie_dialogs_corpus.zip




--2025-04-08 08:49:33--  http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.53
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.53|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip [following]
--2025-04-08 08:49:39--  https://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9916637 (9.5M) [application/zip]
Saving to: ‘cornell_movie_dialogs_corpus.zip’

cornell_movie_dialo  11%[=>                  ]   1.13M   217KB/s    in 5.6s    

2025-04-08 08:49:45 (206 KB/s) - Read error at byte 1183744/9916637 (Connection reset by peer). Retrying.

--2025-04-08 08:49:46--  (try: 2)  https://www.cs.cornell.edu/~cristian/data

In [2]:
import os
import re
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from sklearn.model_selection import train_test_split


In [5]:
import tensorflow as tf
print("GPU disponible :", tf.config.list_physical_devices('GPU'))


GPU disponible : []


In [ ]:
# Chargement des lignes
with open("cornell movie-dialogs corpus/movie_lines.txt", encoding='iso-8859-1') as file:
    lines = file.read().split("\n")

# Dictionnaire id -> ligne
id2line = {}
for line in lines:
    parts = line.split(" +++$+++ ")
    if len(parts) == 5:
        id2line[parts[0]] = parts[4]

# Chargement des conversations
with open("cornell movie-dialogs corpus/movie_conversations.txt", encoding='iso-8859-1') as file:
    conversations = file.read().split("\n")

# Création des paires question-réponse
questions = []
answers = []

for conv in conversations:
    parts = conv.split(" +++$+++ ")
    if len(parts) == 4:
        utterance_ids = eval(parts[3])
        for i in range(len(utterance_ids) - 1):
            if utterance_ids[i] in id2line and utterance_ids[i+1] in id2line:
                questions.append(id2line[utterance_ids[i]])
                answers.append(id2line[utterance_ids[i+1]])


FileNotFoundError: [Errno 2] No such file or directory: 'movie-corpus'

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9?.!,¿]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

questions = [clean_text(q) for q in questions]
answers = [clean_text(a) for a in answers]


In [ ]:
vocab_size = 10000
max_len = 40

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(questions + answers)

question_seq = tokenizer.texts_to_sequences(questions)
answer_seq = tokenizer.texts_to_sequences(answers)

question_pad = pad_sequences(question_seq, maxlen=max_len, padding='post')
answer_pad = pad_sequences(answer_seq, maxlen=max_len, padding='post')


In [ ]:
model = Sequential([
    Embedding(vocab_size, 256, input_length=max_len),
    Bidirectional(LSTM(256, return_sequences=True)),
    Bidirectional(LSTM(256)),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(question_pad, answer_pad, test_size=0.2)

# On prédit le premier mot de la réponse comme approximation simple
model.fit(X_train, np.expand_dims(y_train[:, 0], -1),
          validation_data=(X_val, np.expand_dims(y_val[:, 0], -1)),
          batch_size=128, epochs=10)


In [ ]:
def generate_reply(input_text):
    input_text = clean_text(input_text)
    seq = tokenizer.texts_to_sequences([input_text])
    padded = pad_sequences(seq, maxlen=max_len, padding='post')
    prediction = model.predict(padded)
    word_index = np.argmax(prediction[0])

    for word, index in tokenizer.word_index.items():
        if index == word_index:
            return word
    return "..."

# Exemple :
generate_reply("Are you ready for the adventure?")
